In [ ]:
pip install webdriver_manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Library used to extract and clean data

In [ ]:
from selenium.webdriver import Chrome
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
chrome_driver=ChromeDriverManager().install()
from time import sleep

#to resolve HTTP 403 response
from urllib.request import urlopen, Request

from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Function recalled in the main code to click button in the web pages: Cookies and nextPage - during the automate navigation

In [ ]:
def click_navigate_page(x):
    x.find_element(by=By.ID, value="onetrust-accept-btn-handler").click()
    sleep(2)
    
    try:
        x.find_element(by=By.XPATH, value="//span[@class='SVGInline modal_closeIcon']").click()
        sleep(2)
    except:
        x.find_element(by=By.XPATH, value="//button[@class='nextButton job-search-1iiwzeb e13qs2072']").click()
        sleep(2)
    #nextButton job-search-1iiwzeb e13qs2072 new
    
   # nextButton css-1hq9k8 e13qs2071
    try:
        x.find_element(by=By.XPATH, value="//button[@class='nextButton job-search-1iiwzeb e13qs2072']").click()
        sleep(2)
        prova=1
    except:
        #driver.find_element(by=By.XPATH, value="//button[@class='nextButton css-1hq9k8 e13qs2071']").click()
        prova=2

In [ ]:
# not used 
def string_to_float(x):
        l=[]
        d={}
        for i in range(len(x)):
            l.append(x[i].get_text('span').split('span'))
            if type(l[i][0])==str :
                if re.search("\d.\d+?",l[i][0]) :
                     l[i][0]=float(l[i][0])      
            #d[i]=l[i]
        return l 

def  ad_text(page,text_list_ad,url):
    for j in range(len(page)):
        pro=page[j].get_text('span').split('span')
        #print(pro)
        #print(f'number of ad {j},- {pro[0]} - {pro[1]}')
        for i in range(len(pro)):
            if ((pro[i]=='Italy' or pro[i]=='Lombardy') or (pro[i]=='Emilia-Romagna' or pro[i]=='Apulia') or (pro[i]=='Veneto')):
                #print(f'index number is: {i}, and the value is : {pro[i]}, firm is : {pro[0]} {pro[1]}')
                #print('scrape and get city information from this Ad')
                link = 'https://www.glassdoor.com' + page[j].a['href']
                #print(f'relative firm {pro[0]} {pro[1]} and link= {a2[j].a["href"]}')
                url.append(link)
                url  = link
            
                headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
                req = Request(url=url, headers=headers)
                adPage = urlopen(req).read()
                adPage1=BeautifulSoup(adPage)
            
                lista_Prova=adPage1.get_text('<p>').split('</p>')
                text_list_ad.append(lista_Prova[0].split(' '))
        return text_list_ad

Navigate along the Glassdor page with selenium
- Scrape each AD for each page
- Scrape AD pages to get more insight 

In [ ]:
s=Service('/Downloads/chromedriver_mac64')
driver=Chrome(service=Service(chrome_driver))

#to resolve HTTP 403 response
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
get_url='https://www.glassdoor.com/Job/italy-data-scientist-jobs-SRCH_IL.0,5_IN120_KO6,20.htm?locKeyword=Italy&srs=RECENT_SEARCHES'
req = Request(url=get_url, headers=headers)

html1 = BeautifulSoup(urlopen(req).read())
              
#Number of page to scraper
              
num_page=int(html1.find('div', class_='paginationFooter').get_text('div').split(' ')[-1])

Ad_list = []
lp=[]
list_url=[]
dataFinal=[]
list_detail=[]
loc_index=[] #indice per italy e altri region
name_loc=[] #name location
loc=0
firm=[]            
for numero in range(num_page):
    req = Request(url=get_url, headers=headers)
    html = urlopen(req).read() 
    html_tree = BeautifulSoup(html)
    
    #SCRAPE EACH PAGE
    a=html_tree.find("article")
    a1=a.find('ul')
    a2=a1.find_all('li')#find_all make a list to each tag finded
        
    #dictionaire_detail[1][0]
    
    
    # Create and add values in a list_detail recalling a function'string_to_float'
    
    list_detail.append(string_to_float(a2))
    
    # List of text for each ad with job location Italy
    #Ad_list.append(ad_text(a2,lp,list_url,loc_index,name_loc))
    for j in range(len(a2)):
        pro=a2[j].get_text('span').split('span')
        for i in range(len(pro)):
            if i<5:
                if ((pro[i]=='Italy' or pro[i]=='Lombardy') or (pro[i]=='Emilia-Romagna' or pro[i]=='Apulia') or (pro[i]=='Veneto' or re.search('30d+',pro[i]))):
                    #print(f'index number is: {i}, and the value is : {pro[i]}, firm is : {pro[0]} {pro[1]}')
                    #print('scrape and get city information from this Ad')
                    link = 'https://www.glassdoor.com' + a2[j].a['href']
                    #print(f'relative firm {pro[0]} {pro[1]} and link= {a2[j].a["href"]}')
                    list_url.append(link)
                    url  = link
                    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
                    req = Request(url=url, headers=headers)
                    adPage = urlopen(req).read()
                    adPage1=BeautifulSoup(adPage)
                    if re.search("\d.\d+?",pro[0]):
                        firm.append(pro[1])
                    firm.append(pro[0])
                    lista_Prova=adPage1.get_text('<p>').split('</p>')
                    lp.append(lista_Prova[0].split(' '))
                    loc_index.append(loc)
                    loc+=1
                
                    name_loc.append(pro[i])
    
    #pr.append(string_to_float(a2))
    
    driver=Chrome(service=Service(chrome_driver))
    driver.maximize_window()
    driver.get(get_url)
    sleep(2)
    
    #Selenium istruction to navigate on glassdoor Site, recalling 'click_navigate_page' function
    click_navigate_page(driver)
    
    get_url=driver.current_url

Put in a dictionaire all useful information 

In [ ]:
##CREATE A DICTIONAIRE WHICH WILL BE THE SKELETON FOR THE DB
dictionary_detail={}

g=0
for k in range(len(list_detail)):
    for j in range(len(list_detail[k])):
        dictionary_detail[g]={'firmScore':list_detail[k][j][0], 'firm':list_detail[k][j][1],'ad_Firm_Role':list_detail[k][j][2],'Job_Location':list_detail[k][j][3],'n':list_detail[k][j][4]}
        g+=1

In [ ]:
len(lp)
lp

121

In [ ]:
count=0
for i in range(len(name_loc)):
    if name_loc[i]=='30d+':
        count+=1
count

Create a Data Base 

In [ ]:
column_name=['firm_Score','Firm','n','ad_Firm_Role','Job_Location']
#delete_info(dictionary_detail)
data=pd.DataFrame(dictionary_detail)
data1=data.transpose()
data1.columns=column_name
Tabella1 = data1
Tabella1

,firm_Score,Firm,n,ad_Firm_Role,Job_Location
0,fabbricadigitale,KEY NOT FOUND: ctas.save,Data Scientist,Parma,Easy Apply
1,Nanesa s.r.l.,KEY NOT FOUND: ctas.save,R&D Scientist - Composite Materials,Italy,Easy Apply
2,INSIGHTUP SRL,KEY NOT FOUND: ctas.save,Data Engineer,Porto San Giorgio,€32K - €38K
3,4.2,Manpower,KEY NOT FOUND: ctas.save,Data Scientist,Italy
4,3.7,Umana spa,KEY NOT FOUND: ctas.save,DATA SCIENTIST,Porto San Giorgio
...,...,...,...,...,...
295,4.1,GSK,KEY NOT FOUND: ctas.save,"Director, Pipeline Project Management Vaccines",Italy
296,3.8,Istituto Italiano di Tecnologia,KEY NOT FOUND: ctas.save,Postdoc on Perception and Situational Awarenes...,Genoa
297,3.8,Istituto Italiano di Tecnologia,KEY NOT FOUND: ctas.save,Postdoc on Robot learning and control,Genoa
298,3.8,Istituto Italiano di Tecnologia,KEY NOT FOUND: ctas.save,Postdoc on Legged Locomotion Control,Genoa


Compare and matching Glassdoor city with Elenco comuni IT to find the information about cities in the website

In [ ]:
lista_comuni=pd.read_csv("Elenco-comuni-IT.csv")

In [ ]:
lista_com=lista_comuni.iloc[:,5]
lista_com

In [ ]:
citta=[]
indice=[]
for z in range(len(lp)):
    for i in lista_com:
        for j in lp[z]:
            if(i==j):
                citta.append(i)
                indice.append(z)
                break
    if z not in indice:
        citta.append("smart working")
        indice.append(z)

In [ ]:
Tabella2=pd.DataFrame({'index':indice, 'citta':citta})
a=Tabella2.groupby('index', as_index=False).count()
duplicate_index=a[a['citta']>1].index
max_duplicate=a[a['citta']>1].max()
max_duplicate=max_duplicate[1]

In [ ]:
len(Tabella2)

Remove Duplicate

In [ ]:
ind=[]
for j in range(max_duplicate):
    for i in range(len(Tabella2)): 
        try:
        
            if Tabella2['index'][i]==Tabella2['index'][i+1]:
                ind.append(i+1)
                Tabella2.drop(i+1,inplace=True)
                
            Tabella2.reset_index(inplace=True)
            Tabella2.drop('level_0',axis=1,inplace=True)
            
        except ValueError:
            pass
        except IndexError:
            pass
        except:
            pass
    

In [ ]:
len(Tabella2)

# Data Cleaning

In [ ]:
indexSTR=data1[data1['firm_Score'].str.isnumeric()==False].index
indexSTR=list(indexSTR) #index str values
dataSistemare=data1[data1['firm_Score'].str.isnumeric()==False]

In [ ]:
dataSistemare['Job_Location']=dataSistemare['ad_Firm_Role']
dataSistemare['ad_Firm_Role']=dataSistemare['n']
dataSistemare['n']=dataSistemare['Firm']
dataSistemare['Firm']=dataSistemare['firm_Score']

In [ ]:
dataSistemare['firm_Score']='Null'
dataSistemare

,firm_Score,Firm,n,ad_Firm_Role,Job_Location
0,Null,fabbricadigitale,KEY NOT FOUND: ctas.save,Data Scientist,Parma
1,Null,Nanesa s.r.l.,KEY NOT FOUND: ctas.save,R&D Scientist - Composite Materials,Italy
2,Null,INSIGHTUP SRL,KEY NOT FOUND: ctas.save,Data Engineer,Porto San Giorgio
7,Null,Terishield,KEY NOT FOUND: ctas.save,Data Scientist,Telelavoro
18,Null,Alan Advantage,KEY NOT FOUND: ctas.save,Junior Data Scientist,Roma
25,Null,Titan4,KEY NOT FOUND: ctas.save,Geospatial Data scientist,Roma
26,Null,lol.travel,KEY NOT FOUND: ctas.save,Data scientist,Rimini
29,Null,HESPLORA,KEY NOT FOUND: ctas.save,Junior Data Scientist,Italy
30,Null,HESPLORA,KEY NOT FOUND: ctas.save,Junior Data Scientist,Italy
45,Null,Sacco srl,KEY NOT FOUND: ctas.save,Fermentation Scientist,Como


In [ ]:
data2=data1.drop(labels=indexSTR, axis=0)
data1New=pd.concat([data2, dataSistemare], ignore_index=True)
Glassdoor=data1New[['firm_Score','Firm','ad_Firm_Role','Job_Location']]

Last data set which has within Job_location "Remote" or name of italian region 

In [ ]:
Glassdoor

In [ ]:
Glass_city=Glassdoor[(Glassdoor["Job_Location"]=="Lombardy") | 
                (Glassdoor["Job_Location"]=="30d+") |
                (Glassdoor["Job_Location"]=="Apulia") | 
                (Glassdoor["Job_Location"]=="Italy") | 
                (Glassdoor["Job_Location"]=="Emilia-Romagna") | 
                (Glassdoor["Job_Location"]=="Veneto") |
                (Glassdoor["Job_Location"]=="Easy Apply")]
Glass_city


According to index_y column , merge the two datasets into one, to arrange Job_location

In [ ]:
df_inner = Tabella2.merge(Glass_city, how='inner', left_index=True, right_index=True)
df_inner.drop('Job_Location',axis=1,inplace=True)
df_inner.rename(columns={'index':'index_y'}, inplace=True)
df_inner2=df_inner[['citta','index_y']]

In [ ]:
Glassdoor.rename(columns={'index':'index_y'}, inplace=True)
Glassdoor

In [ ]:
try:
  Glassdoor.reset_index(inplace=True)
except:
  pass

In [ ]:
Final = Glassdoor.merge(df_inner2, how='left', on='index_y')
Final

Clean Column citta and put Job_location values instead of NaN values

In [ ]:
nan=Final[Final['citta'].isnull()].index
Final['citta'][nan]=Final['Job_Location'][nan]

In [ ]:
try:
    Final.drop('Job_Location', axis=1, inplace=True)
except:
    pass

In [ ]:
Final.rename(columns={'citta':'Job_Location'},inplace= True)
try:
    Final.drop('index_y', axis=1, inplace=True)
except:
    pass

#Final DATASET

In [ ]:
Final.to_csv("Glass_Mod.csv")